# test of model misspecification

In [3]:
# Point Python to the path where we have installed the bad and darc packages
import sys
sys.path.insert(0, '/Users/btvincent/git-local/darc-experiments-python')

In [4]:
import pandas as pd
import numpy as np

import darc
from darc.delayed import models
from darc.designs import BayesianAdaptiveDesignGeneratorDARC, DesignSpaceBuilder
# from darc.delayed.designs import Kirby2009, Frye, Griskevicius2011, Koffarnus_Bickel
from dev.darc_parameter_recovery import simulated_experiment_trial_loop
from darc.data_plotting import plot_delay_without_front_end_delays

Test where we just have one model. This will use the model for both response generation (with it's true parameter values) and fit that same model.

In [5]:
D = DesignSpaceBuilder(RA=list(np.arange(5, 99+1, 1)), 
                       DB=list(np.arange(1, 365, 7))).build()
design_thing = BayesianAdaptiveDesignGeneratorDARC(D, max_trials=20)

# create a response model with some true parameters
model_to_fit = models.Hyperbolic(n_particles=5000)
model_to_fit.θ_true = pd.DataFrame.from_dict({'logk': [np.log(0.01)], 'α': [2]})

fitted_model, _ = simulated_experiment_trial_loop(design_thing, model_to_fit)

fitted_model.get_θ_summary_stats('logk')
# plot_delay_without_front_end_delays(ax[row,col], design_thing.get_df())
# hyperbolic_posterior_predictive(ax[row,col], fitted_model)

/Users/btvincent/git-local/darc-experiments-python/bad/optimisation.py:153: RuntimeWarning: invalid value encountered in true_divide
  (n_times_sampled+n_times_sampled_iter))
/Users/btvincent/git-local/darc-experiments-python/bad/optimisation.py:226: RuntimeWarning: invalid value encountered in true_divide
  (n_times_sampled + n_times_sampled_iter))


,logk_entropy,logk_median,logk_mean,logk_lower50,logk_upper50,logk_lower95,logk_upper95
0,-2.054001,-4.634027,-4.638929,-4.658559,-4.62315,-4.707965,-4.581752


This example with generate data from the `Hyperbolic` model, but do parameter estimation on the `Exponential` model throughout the simulated experiment.

In [8]:
D = DesignSpaceBuilder(RA=list(np.arange(5, 99+1, 1)), 
                       DB=list(np.arange(1, 365, 7))).build()
design_thing = BayesianAdaptiveDesignGeneratorDARC(D, max_trials=20)

# create a response model with some true parameters
response_model = models.Hyperbolic(n_particles=5000)
response_model.θ_true = pd.DataFrame.from_dict({'logk': [-4.5], 'α': [2]})

# we will fit with a different class of model
model_to_fit = models.Exponential(n_particles=5000)

fitted_model, _ = simulated_experiment_trial_loop(design_thing, 
                                                  model_to_fit, 
                                                  response_model=response_model,
                                                  track_this_parameter=None)

/Users/btvincent/git-local/darc-experiments-python/bad/optimisation.py:153: RuntimeWarning: invalid value encountered in true_divide
  (n_times_sampled+n_times_sampled_iter))
/Users/btvincent/git-local/darc-experiments-python/bad/optimisation.py:226: RuntimeWarning: invalid value encountered in true_divide
  (n_times_sampled + n_times_sampled_iter))
/Users/btvincent/git-local/darc-experiments-python/darc/delayed/models.py:97: RuntimeWarning: overflow encountered in multiply
  VB = data['RB'].values * self._time_discount_func(data['DB'].values, θ['k'].values)
/Users/btvincent/git-local/darc-experiments-python/bad/choice_functions.py:22: RuntimeWarning: overflow encountered in true_divide
  p_chose_B = θ_fixed['ϵ'] + (1 - 2 * θ_fixed['ϵ']) * _Phi(np.divide(decision_variable, α))


In [9]:
# NOTE THESE ARE ESIMATES OF k (NOT LOGK) FROM THE EXPONENTIAL MODEL
fitted_model.get_θ_summary_stats('k')

,k_entropy,k_median,k_mean,k_lower50,k_upper50,k_lower95,k_upper95
0,-7.376877,0.005491,0.005432,0.005491,0.005491,0.005091,0.005491
